In [1]:
import pydea
import matplotlib as plt
import pandas as pd
import numpy as np

### 根据论文《林业上市公司融资效率的DEA分析》

### 输入数据读取：总资产（X1）、负债总额（X2）、所有者权益（X3）、营业总成本（X4）、筹资活动的现金流入量（X5）

### 输出数据读取：营业收入（Y1） 和净利润（Y2）

### 模型选择：BBC模型

## 输入

In [2]:
stockNum = "600519"

In [3]:
#读取csv文件

data = pd.read_csv("./Assets_and_liabilities/" + stockNum + ".csv", encoding="utf-8")
# 资产总计
total_assets = data.iloc[46, ]
# 负债合计
total_liabilities = data.iloc[80, :]
# 所有者权益
owners_equity = data.iloc[81, :]
# 营业总成本
data_total_operating_costs = pd.read_csv("./Total_operating_costs/000596-profit.csv", encoding="utf-8")
total_operating_costs = data_total_operating_costs.iloc[3, :]
# 筹资活动的现金流入量
data_Funds_inflow_for_financing_activities = pd.read_csv("./Funds_inflow_for_financing_activities/000596-flow.csv", encoding="utf-8")
funds_inflow_for_financing_activities = data_Funds_inflow_for_financing_activities.iloc[32, :]
funds_inflow_for_financing_activities

报表日期        偿还债务支付的现金
20210331     22000000
20201231            0
20200930            0
20200630            0
              ...    
20001231     55200000
20000630     47792000
19991231    108190000
19981231     20000000
19700101            0
Name: 32, Length: 83, dtype: object

## 输出

In [4]:
# 营业收入
operating_income = data_total_operating_costs.iloc[2,:]
# 净利润
profit = data_total_operating_costs.iloc[21,:]
profit

报表日期              五、净利润
20210331    839225582.2
20201231     1847888183
20200930     1532565307
20200630     1006441527
               ...     
19960630      133653539
19951231      147048988
19941231      112961204
19931231      107833958
19700101              0
Name: 21, Length: 94, dtype: object

### 输入输出数据分别合并

In [5]:
# 输入数据合并
inputData = pd.concat([total_assets, total_liabilities, owners_equity, total_operating_costs, funds_inflow_for_financing_activities], axis=1, sort=False)
# inputData.rename(columns={'46':'资产总计','80':'负债合计', "81":'所有者权益', '3':'营业总成本', '32':'偿还债务支付的现金'}) 
inputData = inputData.iloc[1:, :].astype("float64")
inputData = inputData.fillna(value=0)
print(inputData)

outputData = pd.concat([operating_income, profit], axis=1, sort=False)
# outputData = outputData.rename(columns={'2':'营业收入','21':'净利润'})
outputData = outputData.iloc[1:, :].astype("float64")
outputData = outputData.fillna(value=0)
print(outputData)

                    46            80   81            3           32
20210331  2.152180e+11  3.272842e+10  0.0  3.031459e+09  22000000.0
20201231  2.133960e+11  4.567513e+10  0.0  7.878037e+09         0.0
20200930  1.845790e+11  3.040754e+10  0.0  6.048516e+09         0.0
20200630  1.721980e+11  2.967999e+10  0.0  4.162706e+09         0.0
20200331  1.816250e+11  2.588991e+10  0.0  2.443222e+09         0.0
...                ...           ...  ...           ...         ...
19961231  0.000000e+00  0.000000e+00  0.0  2.933809e+08         0.0
19960630  0.000000e+00  0.000000e+00  0.0  0.000000e+00         0.0
19951231  0.000000e+00  0.000000e+00  0.0  5.565298e+08         0.0
19941231  0.000000e+00  0.000000e+00  0.0  4.125243e+08         0.0
19931231  0.000000e+00  0.000000e+00  0.0  4.121197e+08         0.0

[93 rows x 5 columns]
                    2             21
20210331  4.130016e+09  8.392256e+08
20201231  1.029206e+10  1.847888e+09
20200930  8.069421e+09  1.532565e+09
20200630  5.5

## 执行DEA计算，计算综合效率

In [6]:
#执行计算
uni_prob_CRS = pydea.DEAProblem(inputData, outputData, returns='CRS')
myresults_CRS = uni_prob_CRS.solve()

## 执行DEA计算，计算纯技术效率

In [7]:
#执行计算
uni_prob_VRS = pydea.DEAProblem(inputData, outputData, returns='VRS')
myresults_VRS = uni_prob_VRS.solve()

In [8]:
# myresults_CRS['综合效率'] = myresults_CRS['Efficiency']
# myresults_VRS['纯技术效率'] = myresults_VRS['Efficiency']
myresults = pd.DataFrame()
myresults['综合效率'] = myresults_CRS['Efficiency']
myresults['CRS_Status'] = myresults_CRS['Status']
myresults['纯技术效率'] = myresults_VRS['Efficiency']
myresults['VRS_Status'] = myresults_VRS['Status']
myresults['规模效率'] = (myresults_CRS['Efficiency'] / myresults_VRS['Efficiency']).fillna(0)
myresults
# myresults['规模效率']
# myresults

,综合效率,CRS_Status,纯技术效率,VRS_Status,规模效率
20210331,0.0,Optimal,9.062926e-01,Optimal,0.0
20201231,0.0,Optimal,1.000000e+00,Optimal,0.0
20200930,0.0,Optimal,9.974306e-01,Optimal,0.0
20200630,0.0,Optimal,9.855959e-01,Optimal,0.0
20200331,0.0,Optimal,1.000000e+00,Optimal,0.0
...,...,...,...,...,...
19961231,0.0,Optimal,8.124086e-09,Optimal,0.0
19960630,0.0,Infeasible,0.000000e+00,Infeasible,0.0
19951231,0.0,Optimal,3.311397e-09,Optimal,0.0
19941231,0.0,Optimal,3.013876e-09,Optimal,0.0


### 输出数据

In [11]:
# resultData = pd.concat([myresults['Efficiency'], myresults['Status'], myresults['Weights']], axis=1)
# resultData.to_csv("CRS_Result.csv", encoding="utf-8")
# resultData = pd.concat([myresults_CRS['综合效率'], myresults_CRS['CRS_Status'], myresults_VRS['纯技术效率'], myresults_VRS['VRS_Status'], myresults['规模效率']], axis=1)
# resultData = resultData.columns(['综合效率', 'CRS_Status', '纯技术效率', 'VRS_Status', '规模效率'])
# resultData_new = pd.DataFrame(resultData)
# resultData
# resultData.to_csv("CRS_Result" + stockNum + ".csv", encoding="utf-8")
myresults.to_csv("DeaResultFrom" + stockNum + ".csv", encoding="utf_8_sig")